# __시계열 예측 모델 만들기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 10분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_ml/timeseries/timeseries_forecasting.ipynb
- 참고 문서: [DACON 건물별 전력 사용량 데이터 다운로드](https://drive.google.com/uc?export=download&id=1I2sEEOow8AR1lrhNSyHeXTp8hm72PkZF), [Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting](https://arxiv.org/pdf/1912.09363.pdf)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">시계열 분석 이해하기</h4>
    <p>시계열 데이터(time series data)란 시간을 기준으로 측정된 자료를 말합니다. 시계열 자료는 연도별, 분기별, 월별, 일별 또는 시간별 등 시간의 경과(흐름)에 따라 순서대로 관측되는 자료입니다. 그 예로 국내총생산(GDP), 물가 지수, 판매량, 종합주가지수(KOSPI), 강우량, 태양 흑점 수, 실험 및 관측자료 등이 있습니다. 시계열 자료는 연속적으로 측정되는 연속 시계열(continuous time series)과 이산적 시점에서 측정되는 이산 시계열(discrete time series)로 구분할 수 있습니다. 여기서 연속 시계열이란 말 그대로 시간의 모든 점에서 측정된 자료를 의미합니다. 연속 시계열 자료는 모든 시점에서 측정되었기 때문에 분석하기에 부담스러운 데이터 상태입니다. 그래서 현업에서는 이산 시계열 데이터가 주로 사용되어 집니다. 이산 시계열 데이터란 특정한 시점에 측정한 관측값을 의미하며, 일반적으로 관측값 간의 간격을 일정하게 합니다.</p>
</div>

시계열 분석은 일정한 시간 간격으로 표시된 자료의 특성(추세변동, 계절변동, 순환변동, 불규칙변동)을 파악하여 미래를 예측하는 분석 방법입니다. 시계열 분석을 통해서 내년도 판매량, 다음 달 항공기 이용 승객, 앞으로 4개월 동안의 변화, 다음 분기에 예상되는 변화 등을 예측할 수 있습니다. 시계열 분석의 단점은 연구자가 만든 시계열 모형을 이용하여 예측하고자 할 때, 여러 가지 우연한 사건이 발생하여 오차를 일으킬 수 있다는 점입니다. 시계열 예측 모형은 수리적 모형에 불과하여 천재지변, 정치, 경제, 사회, 문화 영역에서 일어나는 여러 변수를 일일이 고려하지 못하기 때문에 오차를 피할 수가 없습니다. 그럼에도 불구하고, 시계열 분석을 통해 얻을 수 있는 고급 정보는 비즈니스가 미래의 변화에 미리 대비 할 수 있도록 하여 여러 분야에서 사용되어지고 있습니다.


__아래는 ThanoSQL 시계열 예측 모델의 활용 및 예시입니다.__

- 온라인 쇼핑몰 내에서 다음 달의 판매량을 미리 예측할 수 있습니다. 고객 데이터를 바탕으로 계절별, 분기별, 시간별 고객들의 구매 특징을 시계열 모델이 학습할 수 있도록 하여 예측값을 바탕으로 맞춤화된 전략 수립을 가능하게 합니다.
- 전력 사용량을 미리 예측할 수 있습니다. 전력 사용량의 계절별 특성을 모델이 학습하여 다음 달 전력 사용량을 미리 예측하고 예측값을 바탕으로 전력 생산량을 조절하여 블랙아웃 현상을 사전에 방지할 수 있습니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 DACON 전력 사용량 예측 AI 경진대회의 빌딩별 전력 사용량 데이터 세트를 사용합니다. 데이터 세트는 60개 건물의 전력 사용량 및 기온, 습도, 풍속 등의 기상 데이터가 포함되어 있습니다. ThanoSQL에서 제공하는 데이터 세트는 시계열 모델을 학습하기 위해 미리 전처리 과정을 거친 데이터로써 학습용 데이터 세트는 2020년 6월 1일부터 2020년 8월 24일, 테스트용 데이터 세트는 2020년 8월 25일부터 2020년 8월 31일까지로 구성되어 있습니다.</p>
</div>

이번 튜토리얼에서는 시계열 예측 모델을 사용하여, 테스트용 데이터 세트의 일주일 전력 사용량을 예측하는 시계열 모델을 만들어 봅니다.

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET electricity_usage_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY elec_usage_train
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/electricity_usage_data/electricity_usage_train.csv'

Success


In [4]:
%%thanosql
COPY elec_usage_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/electricity_usage_data/electricity_usage_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

전기사용량 시계열 예측 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __elec_usage_train__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM elec_usage_train
LIMIT 5

,num,datetime,target,temperature,windspeed,humidity,precipitation,insolation,natural_cooling_sys_flag,solar_sys_flag,...,holiday,before_holiday_flag,days_left_holiday,same_temp_wind_group,mean_target_cluster,mean_target_num,mean_target_stwg,mean_target,log_target,time_idx
0,1,2020-06-01,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.009332,0
1,1,2020-06-01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.004010,1
2,1,2020-06-01,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.000499,2
3,1,2020-06-01,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,8.993279,3
4,1,2020-06-01,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,8.992635,4


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>elec_usage_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>num: 건물 번호</li>
        <li>datetime: 기록 시간</li>
        <li>target: 전력 사용량 (kWh)</li>
        <li>temperature: 기온</li>
        <li>windspeed: 풍속</li>
        <li>humidity: 습도</li>
        <li>precipitation: 강수량</li>
        <li>insolation: 일사량</li>
        <li>natural_cooling_sys_flag: 비전기 냉방 시설 유무</li>
        <li>solar_sys_flag: 태양광 발전 시설 유무</li>
        <li>cluster: 전력 사용 유형</li>
        <li>hour: 시간</li>
        <li>dow: 요일</li>
        <li>date: 날짜</li>
        <li>day: 일</li>
        <li>month: 월</li>
        <li>holiday: 주말, 공휴일</li>
        <li>before_holiday_flag: 휴일 전날 flag (다음날이 휴일이라면 1, 아니라면 0)</li>
        <li>days_left_holiday: 남은 휴일</li>
        <li>same_temp_wind_group: 동일 기온, 풍속인 빌딩 그룹</li>
        <li>mean_target: 전력 사용량 평균</li>
        <li>mean_target_stwg: 동일 기온, 풍속인 빌딩 그룹들의 전력 사용량 평균</li>
        <li>mean_target_num: 건물 별 전력 사용량 평균 </li>
        <li>log_target: 전력 사용량의 로그 치환 값</li>
        <li>time_idx: 시간 순서를 나타내는 인덱스. 시계열 모델 학습에 필요</li>
    </ul>
</div>

## __2. 시계열 예측 모델 생성__

이전 단계에서 확인한 __elec_usage_train__ 테이블을 사용하여 전기사용량 예측 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>elec_predict_model</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 3 min)

In [6]:
%%thanosql
BUILD MODEL elec_predict_model
USING TFT
OPTIONS (
    target_col='target',
    time_idx_col='time_idx',
    group_id_cols=['num'],
    min_encoder_length=1,
    max_encoder_length=840,
    min_prediction_length=1,
    max_prediction_length=168,
    group_normalizer=True,
    time_varying_known_categorical_cols=['num','same_temp_wind_group','holiday','dow','cluster','before_holiday_flag','natural_cooling_sys_flag','solar_sys_flag'],
    time_varying_known_real_cols=['time_idx','hour','temperature','windspeed','humidity','precipitation','insolation','days_left_holiday'],
    time_varying_unknown_real_cols=['target','log_target','mean_target','mean_target_num','mean_target_stwg','mean_target_cluster'],
    validate=False,
    overwrite=True,
    max_epochs=1
    )
AS
SELECT *
FROM elec_usage_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>elec_predict_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>TFT</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"target_col": 시계열 모델의 목푯값이 담겨 있는 컬럼명 (str, default: 'target')</li>
            <li>"time_idx_col": 시간 인덱스 컬럼 (str, default: 'time_idx')</li>
            <li>"group_id_cols": 시계열 식별자 컬럼. 데이터 세트에 여러 시계열이 존재하는 경우 해당 식별자를 리스트 형태로 기재함 (List[str])</li>
            <li>"min_encoder_length": 모델이 예측할 때 사용할 과거 데이터의 최소 길이(time_idx). 기본값은 max_encoder_length 값에 맞춰짐 (int, optional)</li>
            <li>"max_encoder_length": 모델이 예측할 때 사용할 과거 데이터 최대 길이(time_idx) (int, default: 24)</li>
            <li>"min_prediction_length": 최소 예측 길이. 기본값은 max_encoder_length 값에 맞춰짐 (int, optional)</li>
            <li>"max_prediction_length": 최대 예측 길이. 너무 짧지 않도록 선택 (int, default: 6)</li>
            <li>"group_normalizer": GroupNormalizer의 사용 여부 (bool, optional, True|False, default: False)</li>
            <li>"time_varying_known_categorical_cols": 시간이 지남에 따라 변경되고 미래 값이 미리 알려진 범주형 변수 리스트 e.g. 공휴일 (List[str])</li>
            <li>"time_varying_known_real_cols": 시간이 지남에 따라 변화하고 미래에 알려진 연속 변수 리스트 (List[str])</li>
            <li>"time_varying_unknown_real_cols": 시간이 지남에 따라 변경되고 미래에 알려지지 않은 연속형 변수 리스트. 여기에 타겟 값을 포함하는게 좋음 (List[str])</li>
            <li>"validate": 교차검증 사용옵션 (bool, optional, True|False, default: False)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
            <li>"max_epochs": 시계열 예측 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, default: 30)</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 생성된 모델을 사용하여 전력 사용량 예측__

이전 단계에서 생성한 시계열 예측 모델을 사용해 테스트 데이터 세트의 1주일 전력 사용량을 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, <strong>elec_usage_test</strong>)를 사용합니다.

<div class="admonition danger">
<p>시계열 모델을 사용하여 예측하기 위해서는 설정한 인코더 길이 만큼의 정답이 포함된 학습용 데이터를 테스트 데이터 세트 앞에 병합해야 합니다. e.g 만약 모델 인코더 길이를 860으로 설정하고 테스트 데이터의 time_idx 값이 학습용 데이터에 이어 1000부터 시작한다면 time_idx 140 ~ 999의 데이터를 테스트 데이터 세트 앞에 병합해야 합니다.</p>
</div>

In [7]:
%%thanosql
PREDICT USING elec_predict_model
OPTIONS (
        result_col='tft_result'
        )
AS
SELECT
*
FROM elec_usage_test

,num,datetime,target,temperature,windspeed,humidity,precipitation,insolation,natural_cooling_sys_flag,solar_sys_flag,...,before_holiday_flag,days_left_holiday,same_temp_wind_group,mean_target_cluster,mean_target_num,mean_target_stwg,mean_target,log_target,time_idx,tft_result
0,1,2020-07-21,8483.616,22.4,2.3,89.0,0.0,0.0,0.0,0.0,...,0,0.0,2,3374.640000,8504.784,2926.68740,2455.374652,9.045892,1200.0,8549.66
1,1,2020-07-21,8481.024,22.1,1.4,91.0,0.0,0.0,0.0,0.0,...,0,0.0,2,3374.640000,8504.784,2926.68740,2455.374652,9.045586,1201.0,8551.31
2,1,2020-07-21,8444.088,22.1,1.9,89.0,0.0,0.0,0.0,0.0,...,0,0.0,2,3374.640000,8504.784,2926.68740,2455.374652,9.041222,1202.0,8552.14
3,1,2020-07-21,8429.184,22.1,0.9,89.0,0.0,0.0,0.0,0.0,...,0,0.0,2,3374.640000,8504.784,2926.68740,2455.374652,9.039455,1203.0,8552.60
4,1,2020-07-21,8427.240,22.0,1.5,90.0,0.0,0.0,0.0,0.0,...,0,0.0,2,3374.640000,8504.784,2926.68740,2455.374652,9.039225,1204.0,8553.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-07-27,3576.528,24.4,1.9,94.0,0.0,0.0,1.0,1.0,...,0,0.0,1,2072.407524,3332.196,2269.20645,2397.927902,8.182148,1363.0,1267.99
10076,60,2020-07-27,3573.936,24.0,2.1,93.0,0.0,0.0,1.0,1.0,...,0,0.0,1,2072.407524,3332.196,2269.20645,2397.927902,8.181423,1364.0,1267.74
10077,60,2020-07-27,3494.448,23.8,2.7,89.0,0.0,0.0,1.0,1.0,...,0,0.0,1,2072.407524,3332.196,2269.20645,2397.927902,8.158931,1365.0,1267.89
10078,60,2020-07-27,3329.424,23.8,1.3,90.0,0.0,0.0,1.0,1.0,...,0,0.0,1,2072.407524,3332.196,2269.20645,2397.927902,8.110555,1366.0,1268.37


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>elec_predict_model</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __4. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다.

In [8]:
%%thanosql
EVALUATE USING elec_predict_model
AS
SELECT
*
FROM elec_usage_test

,metric,score
0,MAE,1787.217024
1,MAPE,116.790000
2,SMAPE,67.510000


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>EVALUATE USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>elec_predict_model</strong> 모델을 평가합니다.</li>
    </ul>
</div>

## __5. 튜토리얼을 마치며__

이번 튜토리얼에서는 건물별 전력 사용량 데이터 세트를 사용하여 시계열 예측 모델을 만들어 보았습니다. 이번 튜토리얼은 성능보다는 빠른 작동위주로 진행하였습니다. 좀 더 나은 성능을 위해서는 데이터 세트에 최적화하여 학습 횟수를 늘리는 등의 정밀한 파라미터 튜닝이 필요합니다. 추가적인 데이터 전처리 과정 또한 모델의 성능을 향상 시킬 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>시계열 예측 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>